In [115]:
import numpy as np
import random as rand

def check_optimality(gk,Hk,tolT): 
#   Revisa que se cumplan las condiciones de optimalidad: Grad=0, Hess positiva semidefinida

#   gk - Vector Gradiente de f en xk
#   Hk - Matriz Hessiana de f en xk
#   tolT - Tolerancia de el gradiente
    print(gk)
    return es_pos_def(Hk) and all(abs(gk)<tolT)
def Grad(f, xk, h):
#   Calcula el gradiende de f en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n = xk.size
    g = np.zeros(n)
    for i in range(0, n):
        b = np.copy(xk)
        b[i] += h
        g[i] = (f(b) - f(xk))/(h)
    return g

def Hess(f,xk,h):
#   Calcula el Hessiano en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n=xk.size
    H=np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            ff=np.copy(xk)
            ff[i]+=h
            ff[j]+=h
            
            fb=np.copy(xk)
            fb[i]+=h
            fb[j]-=h
            
            bf=np.copy(xk)
            bf[i]-=h
            bf[j]+=h
            
            bb=np.copy(xk)
            bb[i]-=h
            bb[j]-=h
            
            H[i,j]=(f(ff)-f(fb)-f(bf)+f(bb))/(4*h**2)
    return H

def es_pos_def(A):
#   Prueba si la matriz A es positiva semidefinida si A es simétrica

#   A - Matriz a probar

        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
        return False
def PruebaMod_Hess(Hk, Beta ) :
#   Modifica la Hessiana para que sea positiva semidefinida, según el algoritmo 3.3 de Nocedal Ed. 2

#   Hk - Matrizz hessiana a modificar
#   Beta - Valor arbitrario a aumentar

    n=int(np.sqrt(np.size(Hk)))
    tk=0
    v=np.matrix.diagonal(Hk)
    
    if min(v)<0:
        tk=-min(v)+Beta    
    

    while es_pos_def(Hk+np.eye(n)*tk)==False:
           tk= max(2*tk, Beta);
    
    return Hk+np.eye(n)*tk
     
def BactrackSearch(f,xk,pk,gk,alpha0,c,rho):
#   Busca el paso según el algoritmo de 3.1 de Nocedal Ed. 2

#   f - Función a evaluar
#   xk - punto alrededor del que se evalúa
#   pk -Dirección elegida previamente
#   alpha0 - Valor de paso máximo
#   c - peso sobre la derivada direccional
#   rho - factor de disminución del paso

    alpha_k=alpha0
   

    while f(xk+alpha_k*pk)> f(xk)+c*alpha_k*np.dot(gk,pk):
        
        alpha_k=alpha_k*rho
    return alpha_k
def fi(f,xk,pk):
    return lambda a: f(xk + a*pk)

def Inter(l, b):
    m = (l+b)/2
    return m

def Zoom(alo, ahi, f, xk, pk, c1, c2,  h, maxiter):
    
    a =np.zeros(maxiter+1)
    fi_gorro = fi(f, xk, pk)
    fi0 = fi_gorro(0)
    fid0 = Grad(fi_gorro, np.zeros(1), h)
    ac = 0
    found = False


    for i in range(0,maxiter):
        
        a[i] = Inter(alo, ahi)
        fid = Grad(fi_gorro, np.array([ a[i] ]), h)
        fie = fi_gorro(a[i])    
    
        if (fie>fi0 + c1*a[i]*fid0):
            ahi = a[i]    
            
        else:
            if abs(fid)<= -c2*fid0:
                ac = a[i]
                found = True
                break
            if fid*(ahi-alo)>= 0:
                ahi = alo
            alo = a[i]

    if not found:
        print("No convergimos en la interpolacion con {} max iteraciones".format(maxiter))
        ac = a[i]

    return ac  

def LineSearchWolf(f,a_max, xk, pk, c1, c2, h, maxiter):
    
    
    a =np.zeros(maxiter+1)
    fi_gorro = fi(f,xk,pk)
    fi0 = fi_gorro(0)
    fid0 = Grad(fi_gorro, np.zeros(1), h)
   
    for i in range(0,maxiter):
        
        a[1]= rand.uniform(a[i-1], a_max)
        
        fie = fi_gorro(a[i])
        fid = Grad(fi_gorro, np.array([ a[i] ]), h)
        
        
        if fie > (fi0 + c1*a[i]*fid0):
            a[i+1] = Zoom(a[i-1],a[i], f, xk, pk,c1, c2, h, maxiter)
            break
        
        if abs(fid) <= -c2*fid0:
            a[i+1] = a[i] 
            break
                    
        if fid >= 0:
            a[i+1] = Zoom(a[i],a[i-1], f, xk, pk, c1, c2, h, maxiter)
            break
  
    return a[i+1]    
    
    
    
    
    
    
    
class Optimaizer3000:
    def __init__(self,f,x0):
        self.f=f
        self.x0=x0

   
 #----------------------------------------------------------------------------------------------------------------------------   
    
    
    
    
    def Min_LineSearchNewtonHessMod(self, tolT , h1 ,h2 , alpha0, c, rho,maxit):  
        x0=self.x0
        f=self.f
    #   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
    #   con una aproximación de la Hessiana adaptada para garantizar que sea positiva definida.

    #   f-Función
    #   x0 - Punto inicial
    #   tolT -  Tolerancia sobre condición de optimalidad
    #   h1 -Diferencias para gradiente
    #   h2 -Diferencias para Hessiana
    #   c - peso sobre el gradiente para búsqueda lineal Bactracking
    #   rho - factor de disminución del paso para búsqueda lineal Bactracking
    #   maxit - Máximo número de Iteraciones

        xk=x0
        n=np.size(x0)
        for k in range(0,maxit):
    
    
            gk=Grad(f,xk,h1)                                       
            Hk=Hess(f,xk,h2)         
        
            if check_optimality(gk,Hk,tolT) :
                break
        
            HessMod=PruebaMod_Hess( Hk, 1e-3 ) 

            pk= -np.linalg.solve(HessMod,gk)
        
        
            alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
            xk_1=xk
            xk=xk+alpha_k*pk
        
        
            print(xk)
            print(f(xk))
            
        return [f(xk),xk,k]
    
    
 #-----------------------------------------------------------------------------------------------------------------------------   
    
   
    def Min_LineSearchNewton(self , tolT , h1 ,h2 , alpha0, c, rho,maxit):  
        
    #   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
    #   con una aproximación de la Hessiana adaptada

    #   f-Función
    #   x0 - Punto inicial
    #   tolT -  Tolerancia sobre condición de optimalidad
    #   h1 -Diferencias para gradiente
    #   h2 -Diferencias para Hessiana
    #   c - peso sobre el gradiente para búsqueda lineal Bactracking
    #   rho - factor de disminución del paso para búsqueda lineal Bactracking
    #   maxit - Máximo número de Iteraciones

        x0=self.x0
        xk=x0
        f=self.f
        n=np.size(x0)
        for k in range(0,maxit):
    
    
            gk=Grad(f,xk,h1)                                       
            Hk=Hess(f,xk,h2)         
        
            if check_optimality(gk,Hk,tolT) :
                break
        
            pk= -np.linalg.solve(Hk,gk)
        
        
            alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
            xk_1=xk
            xk=xk+alpha_k*pk
        
        
            print(xk)
            print(f(xk))
            
        return [f(xk),xk,k]
    
    
 #--------------------------------------------------------------------------------------------------------------------------   
    
    def BFGS(self,epsilon,maxit,h1,h2):
        x0=self.x0
        f=self.f
    #Busca el mínimo de la función con un método Quasinewton y una matriz aproximada con BFGS 
    #   f- Función
    #   x0- Punto inicial
    #   epsilon- Tolerancia para la condición del gradiente
    #   h1- diferencia para la primera aproximación de Hessiana
    #   h2- diferencia para la aproximación del Gradiente

        n=np.size(x0)
        Hk=np.linalg.inv(Hess(f,x0,h1))
        gk=Grad(f,x0,h2)
        print(gk)
        xk=x0
        for k in range(0 ,maxit):
            if all(abs(gk)<epsilon):
                break
            pk=np.matmul(-Hk,gk)
            
            #alphak=BactrackSearch(f,xk,pk,gk,1,0.8,0.9)
            alphak= LineSearchWolf(f,1, xk, pk, 0.8, 0.9, 0.00001, 100)
            
            xk_1=alphak*pk+xk
            gk_1=Grad(f,xk_1,h2)
            sk= xk_1-xk
            yk= gk_1-gk
        
            rhok=1/np.dot(yk,sk)
            print(rhok)    
            Hk= np.matmul(np.matmul((np.identity(n)-rhok*np.matmul(sk,np.transpose(yk))),Hk),(np.identity(n)-rhok*np.matmul(yk,np.transpose(sk))))
            gk=gk_1
            xk=xk_1
            print(xk)
            print(f(xk))

        return [f(xk),xk,k]
#--------------------------------------------------------------------------------------------------------------------------    

    def NewtonAlgorithm(self,tolT , h1 ,h2 ,maxit):  
        x0=self.x0
        f=self.f
    #   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de Newton sin ninguna modificación o 
    #   procedimiento extra.

    #   f-Función
    #   x0 - Punto inicial
    #   tolT -  Tolerancia sobre condición de optimalidad
    #   h1 -Diferencias para gradiente
    #   h2 -Diferencias para Hessiana
    #   c - peso sobre el gradiente para búsqueda lineal Bactracking
    #   rho - factor de disminución del paso para búsqueda lineal Bactracking
    #   maxit - Máximo número de Iteraciones

        xk=x0
        n=np.size(x0)
        for k in range(0,maxit):
    
    
            gk=Grad(f,xk,h1)                                       
            Hk=Hess(f,xk,h2)         
        
            if check_optimality(gk,Hk,tolT) :
                break
        
            pk= -np.linalg.solve(Hk,gk)
        
        
            xk_1=xk
            xk=xk+pk
        
        
            print(xk)
            print(f(xk))
            
        return [f(xk),xk,k]

In [207]:
def rosenbrock1_100(v):
    # a=1 b=100
    return ((1-v[0])**2+100*(v[1]-v[0]**2)**2)

def costo_camaras(c):
    
    
    global data
    global N
    global M
#Calcula el Costo de posicionar un número N de cámaras para cubrir M crimen. Hay uncosto cuadrático para la distancia 
#entre una cámara y un crimen. Y también hay un costo inverso a la distancia al cuadrado entre cámaras. Lo que implica
#una serie de singularidades para todos los puntos donde dos cámaras son iguales, por lo que esta no puede ser condición 
#inicial.

    costo=0

#   c-Lista de arrays, que indican la localización de las cámaras,   N arrays
#   data-Lista de arrays, que indican la localicación de los crímenes, M arrays
#   delta= peso de distancia camaras crimenes
#   sigma= peso del inverso de la distancia camara camara
    
    #Primero el costo de distancia de cámaras a crímenes
    for i in range(0,2*N,2):
        for j in range(0,M-1):
            costo=(c[i]-data[j][0])**2+(c[i+1]-data[j][1])**2
   
    #Luego el costo de cercanía de las cámaras
    for i in range(0,2*N,2):
        for j in range(0,2*N,2):
            if i!=j:
                costo=costo+1/((c[i]-c[j])**2+(c[i+1]-c[j+1])**2)
    
    #Regresamos finalmente el costo total:
    
    return costo

import csv
#En este script obtenemos los datos de nuestra función

with open('crime.csv', 'r') as file:
    reader = csv.reader(file)
    n=31057

    rows=[]
    data=[]
    for row in reader: 
        rows.append(row)

for i in range(1,M):
    data.append(np.asarray(rows[i][3:],dtype=float))

N=800    
M=1305    

c0=np.zeros(2*N)
for i in range(0,2*N):
    c0[i]=rand.uniform(0, 10)
c0=np.array(c0)
    
#-----------------------------------------------------------------------------------------------------------------------------    
    

In [208]:
Camaritas=Optimaizer3000(costo_camaras,c0)
Optimaizer3000.NewtonAlgorithm(Camaritas,0.0000000001 , 0.0000001,0.0000001 ,100)

KeyboardInterrupt: 

In [149]:
Rosenbrock=Optimaizer3000(rosenbrock1_100,np.array([0.799999448753295, 5.450318174756265]))
print(Rosenbrock)

Optimaizer3000.NewtonAlgorithm(Rosenbrock,0.0000000001 , 0.0000001,0.0000001 ,100)

[-1539.70110659   962.06382295]
[0.82293267 0.51020649]
2.8206432079780153
[ 54.62164518 -33.40232743]
[0.8280573 0.6856561]
0.029564344705810208
[-0.33630828 -0.00454862]
[0.99919177 0.96909712]
0.08577394419920122
[11.70378695 -5.8574055 ]
[0.99930366 0.99860772]
4.848958253062731e-07
[-1.32244910e-03 -5.10801859e-06]
[0.99996997 0.99993946]
9.259212206043262e-10
[ 1.77633485e-04 -8.87962169e-05]
[0.99996995 0.99993985]
9.031625551362875e-10
[-6.50355247e-12 -3.33064274e-14]


[9.031625551362875e-10, array([0.99996995, 0.99993985]), 6]

In [119]:
Optimaizer3000.BFGS(Rosenbrock,0.00000001,20,0.0000001,0.0000001)

[-1539.70110659   962.06382295]
0.0038741266522512533
[0.80523474 4.32256813]
1349.9868813964183
0.003768085189103711
[1.67967717 4.21615985]
195.0210668062314
0.02371975745103925
[1.56652642 3.35883959]
82.19351002331027
0.13126312091683495
[1.62912866 3.34837312]
48.60284626129392
inf
[1.62912866 3.34837312]
48.60284626129392
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan


<ipython-input-115-1c676a756194>:310: RuntimeWarning: divide by zero encountered in double_scalars
  rhok=1/np.dot(yk,sk)
<ipython-input-115-1c676a756194>:312: RuntimeWarning: invalid value encountered in double_scalars
  Hk= np.matmul(np.matmul((np.identity(n)-rhok*np.matmul(sk,np.transpose(yk))),Hk),(np.identity(n)-rhok*np.matmul(yk,np.transpose(sk))))


[nan, array([nan, nan]), 19]

In [112]:
Optimaizer3000.Min_LineSearchNewton(Rosenbrock, 0.000001, 0.000001,0.000001 , 1, 0.9, 0.9,100)

[-1539.70174415   962.06391117]
[0.79999257 4.55883447]
1535.7756766334699
[-1254.41973387   783.76937086]
[0.79984108 3.83274442]
1019.5641124471894
[-1021.95730312   638.59983356]
[0.79972798 3.24107304]
676.8245986862624
[-832.60036922  520.30173936]
[0.7996859  2.75888099]
449.2187500080651
[-678.33722136  423.87679173]
[0.79959047 2.36603716]
298.1867735086783
[-552.65957133  345.3385483 ]
[0.79948349 2.04591213]
197.93146786201302
[-450.26666945  281.34775755]
[0.79935059 1.78502349]
131.38609943137786
[-366.84344568  229.21252483]
[0.79919422 1.57239447]
87.2167305887814
[-298.87918656  186.7367141 ]
[0.79899354 1.39906027]
57.902226870934925
[-243.50994567  152.134019  ]
[0.79874561 1.25771165]
38.44543137271033
[-198.40090197  123.94351942]
[0.79844156 1.14239203]
25.531321815360716
[-161.65082585  100.97672305]
[0.79806645 1.04823792]
16.95983859817449
[-131.71048125   82.26567344]
[0.79760519 0.97128206]
11.270702713807806
[-107.31816156   67.02170563]
[0.79703798 0.90828082

[0.09749407418777994, array([0.73462691, 0.55613002]), 99]

In [113]:
Optimaizer3000.Min_LineSearchNewtonHessMod(Rosenbrock, 0.000001, 0.000001,0.000001 , 1, 0.9, 0.9,100)

[-1539.70174415   962.06391117]
[1.84460236 5.24248789]
339.2475995710627
[-1355.8855029   367.9861033]
[2.8289581  7.96442812]
3.4938971692270666
[47.31288334 -7.71506502]
[2.78571832 7.72772353]
3.2944347013708457
[39.79227602 -6.50050839]
[2.73666692 7.46112892]
3.0956314945945453
[34.36446808 -5.64328532]
[2.68279345 7.17207095]
2.8958519790534054
[30.52877339 -5.06184421]
[2.62560691 6.87044275]
2.6972082872374394
[27.7969773  -4.67367661]
[2.56656246 6.56519908]
2.5027107018023758
[25.76644767 -4.4086535 ]
[2.50688458 6.26340377]
2.3150809517677065
[24.14081998 -4.21320364]
[2.44731856 5.96909084]
2.1358478142578825
[22.74702312 -4.05535595]
[2.38834065 5.68459041]
1.9658300369558888
[21.48509259 -3.91603413]
[2.33013964 5.41061413]
1.8051310255429074
[20.3124437  -3.78722405]
[2.27286436 5.14759395]
1.6537402197565372
[19.20193597 -3.66358898]
[2.21660806 4.89563952]
1.5115058368617398
[18.13920437 -3.54225344]
[2.16140875 4.65457561]
1.3781529540984754
[17.11925629 -3.42233587]

[6.864944491046329e-06, array([1.00251791, 1.0049697 ]), 99]

In [181]:
data[13][1]
c0[15999]



9.79125155141025